In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from bs4 import BeautifulSoup
import requests
import time
from googletrans import Translator

In [2]:
def connect_url(target_url):
    """
    対象のURLにアクセスする関数
    アクセスできない等のエラーが発生したら例外を投げる
    """
    # 接続確立の待機時間、応答待機時間を10秒とし、それぞれの値を超えた場合は例外が発生（ConnectTimeout）
    data = requests.get(target_url, timeout=10)
    data.encoding = data.apparent_encoding
    # アクセス過多を避けるため、2秒スリープ
    time.sleep(2)

    # レスポンスのステータスコードが正常(200番台)以外の場合は、例外を発生させる(HTTPError)
    if data.status_code == requests.codes.ok:
        return data
    else:
        data.raise_for_status()

In [3]:
url = 'https://www.tripadvisor.com/Restaurant_Review-g187147-d10085290-Reviews-Kodawari_Ramen_Yokocho-Paris_Ile_de_France.html'
response = connect_url(url)


In [4]:
soup = BeautifulSoup(response.text, 'lxml')

In [5]:
reviews = []
review_container = soup.find_all(class_='review-container')
for i in range(len(review_container)):
    review = review_container[i].find_all("p", class_='partial_entry')[0].text
    reviews.append(review)

In [6]:
translator = Translator()
translator.translate("Hello", dest='ja')

In [9]:
translator = Translator(service_urls=['translate.googleapis.com'])
translated = translator.translate("Der Himmel ist blau und ich mag Bananen", dest='en')
print(translated.text)

The sky is blue and I like bananas


In [13]:
reviews_translated = []
translations = translator.translate(reviews, dest='ja')
for translation in translations:
    reviews_translated.append(translation.text)

In [19]:
zipped = zip(reviews, reviews_translated)

In [20]:
zipped[0]

TypeError: 'zip' object is not subscriptable

In [17]:
trip_advisor_reviews_df = pd.DataFrame([reviews, reviews_translated], columns = ["en", "jp"])

ValueError: 2 columns passed, passed data had 10 columns

In [16]:
trip_advisor_reviews_df

,0,1,2,3,4,5,6,7,8,9
0,We didn't get to go to Japan like we usually d...,"Ambiance is great. The food however, needs som...",This is a small ramen restaurant. I waited so ...,"I’ve been there many time, I tried pretty much...","We arrived for the opening time of 12h00, wait...",We decided to try due to the long queue we saw...,Worth the wait! Had to queue for about 1 hour ...,"Really excellent food, and fab service. Also r...",The atmosphere of this restaurant was great an...,"The best ramen I’ve ever had. Tender meat, fre..."
1,毎年のように日本に行くことができなかったので、超高品質の日本食を見逃していました。お正月にパ...,雰囲気は素晴らしいです。ただし、料理には調味料が必要です。ラーメンは実際に嫌なほど当たり障り...,小さなラーメン屋です。冬の間、私は長い間家族と並んで待っていたので、実際に通りの反対側にある...,私はそこに何度も行っており、さまざまな種類のラーメンを試しました。\n全部好き。場所は本当に...,私たちは 12 時の開店時間に到着し、待ち行列で 25 分待ってから入りました。最終的に入る...,ある夜、オンラインで良いレビューを見て長い列ができていたので、試してみることにしました。食べ...,待つ価値あり！土曜日の夜に約 1 時間待ち行列に入れなければなりませんでした。本物の日本の雰...,本当に素晴らしい料理と素晴らしいサービス。また、他のパリのレストランと比較して比較的安価です...,このレストランの雰囲気は素晴らしく、すべての従業員はとても快適でした。唯一の問題は、彼らのラ...,私が今まで食べた中で最高のラーメン。柔らかい肉、新鮮な食材、素晴らしいスープ、まともなバリエ...


In [0]:
# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

your_trip_advisor_df = ... # Compute a Pandas dataframe to write into your_trip_advisor


# Write recipe outputs
your_trip_advisor = dataiku.Dataset("your_trip_advisor")
your_trip_advisor.write_with_schema(your_trip_advisor_df)